In [122]:
# -*- coding: utf-8 -*-
# python 3.6
from __future__ import print_function

import sys
import re
import os
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import pprint
import datetime
# importing necessary packages

example_table_link = "https://jobs.mo.gov/content/missouri-warn-notices-py-2017"


def remove_control_chart(s):
    """
    :param s: string that may not be utf-8 encode
    :return: 
    """
    s = s.replace('\xa0', '')
    s = re.sub(r'\s\s', ' ', s)
    s = '"' + s + '"'
    return s


def to_string(s):
    """
    makes a string
    input example: 10000
    output example: '1000'
    """
    try:
        return str(s)
    except:
        # Change the encoding type if needed
        return s.encode('utf-8')


def clean_up_cols(s):
    """
    :param s: string for example # AFFECTED
    :return: space removed for example #_affected
    """
    s = to_string(s)
    s = s.lower()
    s = s.replace(' ', '_')
    s = s.replace('#', 'number')
    s = '"' + s + '"'
    return s


def WEBSCRAPeTABLES(link, outputfile_name):
    """
    :param link: this will be the link where you want to scrape table
    :return: json and csv that returns metadata or table data with header keys
    """
    keys_ls = []
    with urllib.request.urlopen(link) as response:
        fout = open(outputfile_name,'w')
        the_page = response.read()
        bs = BeautifulSoup(the_page,features="lxml")
        table = bs.find(lambda tag: tag.name == 'table')
        tabletr = table.tr
        print("RAW XML HEADERS:")
        pprint.pprint(tabletr)
        list_columns = tabletr.find_all('th')
        print("\nCLEANED HEADERS:")
        # this provides the list of columns using the clean_up_cols fxn defined above
        colnames = [clean_up_cols(lc.text) for lc in list_columns]
        header = ','.join(colnames)
        print(header)
        fout.write(header)
        rows = table.findAll(lambda tag: tag.name == 'tr')
        data = []
        counter = 0
        print("\nROW CONTENTS:")
        for row_content in rows:
            counter += 1
            values = row_content.find_all('td')
            csv_row = ','.join([(remove_control_chart(to_string(val.text))) for val in values])
            print(csv_row)
            fout.write(csv_row + 
                      '\n')
    print("\n\n*******\n\nwrote out", to_string(counter), "rows from table\n in csv file called " + outputfile_name)

name = to_string(example_table_link.split('/')[-1]) + ".csv"
WEBSCRAPeTABLES(example_table_link, name)





RAW XML HEADERS:
<tr><th scope="col"><span>DATE RECEIVED</span></th>
<th scope="col"><span>COMPANY NAME</span></th>
<th scope="col"><span>LOCATION</span></th>
<th scope="col"><span>COUNTY</span></th>
<th scope="col"><span>REGION</span></th>
<th scope="col"><span>TYPE</span></th>
<th scope="col"><span>LAYOFF DATE</span></th>
<th scope="col"><span># AFFECTED</span></th>
</tr>

CLEANED HEADERS:
"date_received","company_name","location","county","region","type","layoff_date","number_affected"

ROW CONTENTS:

"07/06/17","SunEdison, Inc.","St. Louis","St. Louis County","St. Louis County","Layoff","07/10/2017","6"
"07/24/17","Neuterra d/b/a Fulton Medical Center ("Hospital")","Fulton","Callaway","Central","Closing","09/22/2017","158"
"07/31/17","Lozier Corporation","Union","Franklin","Jefferson/Franklin Consortium","Closing","10/06/2017 -  Fall 2017","92"
"08/03/17","Diodes Fabtech, Inc.","Lee's Summit","Jackson","Kansas City & Vicinity","Closing","10/06/2017","167"
"08/16/17","Frontier Commu

In [123]:
# now checking to see how csv filecomesin 

missouri=pd.read_csv("missouri-warn-notices-py-2017.csv")
missouri.head()

,date_received,company_name,location,county,region,type,layoff_date,number_affected
0,07/06/17,"SunEdison, Inc.",St. Louis,St. Louis County,St. Louis County,Layoff,07/10/2017,6
1,07/24/17,"Neuterra d/b/a Fulton Medical Center (Hospital"")""",Fulton,Callaway,Central,Closing,09/22/2017,158
2,07/31/17,Lozier Corporation,Union,Franklin,Jefferson/Franklin Consortium,Closing,10/06/2017 - Fall 2017,92
3,08/03/17,"Diodes Fabtech, Inc.",Lee's Summit,Jackson,Kansas City & Vicinity,Closing,10/06/2017,167
4,08/16/17,Frontier Communications Corporation,Weldon Spring,St. Charles,St. Charles County,Closing,10/16/2017 - 10/20/2017,141


In [125]:
missouri.tail()
# here if you would like you can drop the last column -- wanted to keep to see how these would parse

,date_received,company_name,location,county,region,type,layoff_date,number_affected
34,06/15/18,Hard Rock Cafe - St. Louis,St. Louis,St. Louis City,St. Louis City,Closing,"August 16, 2018",57
35,06/15/18,syncreon U.S.,Kansas City,Kansas City,Kansas City & Vicinity,Layoff,"August 17, 2018",207
36,06/28/18,State Farm Mutual Automobile Insurance Company...,Earth City,St. Louis County,St. Louis County,Closing,"August 31, 2018",136
37,06/29/18,"American Airlines, Inc. - St. LouisPilot Crew ...",St. Louis,St. Louis,St. Louis,Closing,"September 3, 2018",155
38,NaN,NaN,NaN,NaN,NaN,NaN,TOTAL,6137


In [129]:
two_table_link = "http://genelex.com/clinical-guidance/cardiology"

WEBSCRAPeTABLES(two_table_link, outputfile_name="drugCardiologyCyp.csv")

RAW XML HEADERS:
<tr><th>Drug</th><th>Biomarker</th><th>Drug</th><th>Biomarker</th></tr>

CLEANED HEADERS:
"drug","biomarker","drug","biomarker"

ROW CONTENTS:

"carvedilol","CYP2D6","propafenone","CYP2D6"
"clopidogrel","CYP2C19","propranolol","CYP2D6"
"isosorbide and hydralazine","NAT1;NAT2","quinidine / dextromethorphan","CYP2D6"
"metoprolol","CYP2D6","ticagrelor","CYP2C19"
"prasugrel","CYP2C19","warfarin","CYP2C9 and VKORC1"


*******

wrote out 6 rows from table
 in csv file called drugCardiologyCyp.csv


In [130]:
# issue is
drugCYP = pd.read_csv("drugCardiologyCyp.csv")
drugCYP

,drug,biomarker,drug.1,biomarker.1
0,carvedilol,CYP2D6,propafenone,CYP2D6
1,clopidogrel,CYP2C19,propranolol,CYP2D6
2,isosorbide and hydralazine,NAT1;NAT2,quinidine / dextromethorphan,CYP2D6
3,metoprolol,CYP2D6,ticagrelor,CYP2C19
4,prasugrel,CYP2C19,warfarin,CYP2C9 and VKORC1


In [ ]:
# unfortunately this code is too basic and needs a little work to grab more tables from a webpage